# Creating an interactive choropleth map using Bokeh  

Here I'm generating an interactive county-level map of population data using Boken.

This seems pretty labor intensive but it is mainly due to the processing of a couple shape files. This only needs to be done once and can then can be used to make any county-level map.

In [1]:
import os
import itertools
import shapefile
import datetime
import requests
import zipfile

import pandas as pd
import pandas.core.algorithms as algos
import numpy as np

# Bokeh ouput related
from bokeh.io import show, output_notebook, save, output_file

# Various color schemes
from bokeh.palettes import Greys256, Magma256, Inferno256, Plasma256, linear_palette, Blues, GnBu9, viridis
from  data import my_colors # My custom color scheme
import colorcet as cc # https://colorcet.pyviz.org/

# A bunch of Bokeh chart objects
from bokeh.plotting import figure
from bokeh.layouts import row, gridplot
from bokeh.models.widgets import Paragraph
from bokeh.models import (ColumnDataSource,
                          HoverTool,
                          LogColorMapper,
                          ColorBar,
                          LogTicker,
                          LinearColorMapper,
                          NumeralTickFormatter,
                          FixedTicker,
                          Label)

try:
    from StringIO import StringIO
except ImportError:
    from io import BytesIO as StringIO


output_notebook()

path = os.getcwd()

Loading BokehJS ...

#### Read in the population data

In [2]:
col_to_keep = ["state_abbrev", "fips_combined", "county_shortnm", "population"]

pop_data_types = {"state_abbrev": str,
                  "fips_combined": str,
                  "county_shortnm": str,
                  "population": int}

pop = pd.read_csv(path + '/data/population.csv', usecols=col_to_keep, dtype=pop_data_types)

# Ensure the fips state and county combined codes are of length 5, padded with zeros.
pop['fips_combined'] = pop.fips_combined.apply(lambda x: x.zfill(5))

pop.head()

,state_abbrev,fips_combined,county_shortnm,population
0,CA,06075,San Francisco,884
1,MA,25025,Suffolk,797
2,NE,31007,Banner,0
3,NC,37181,Vance,44
4,TX,48421,Sherman,3


#### Bin the population data

In [3]:
# Bin the data into 256 intervals
bins = algos.quantile(np.unique(pop.population), np.linspace(0, 1, 257))
np_bins = pd.cut(pop.population, bins, include_lowest=True, retbins=False, labels=[x + 1 for x in range(256)])
np_bins.rename("bin", inplace=True)

# Join bins back to the original dataset
pop_bins = pd.concat([pop, np_bins], axis=1)

# Get the max value in each bin; these values will be used to create the legend
pop_bin_max = pop_bins.groupby("bin")['population'].max().to_dict()

pop_bins.head()

,state_abbrev,fips_combined,county_shortnm,population,bin
0,CA,06075,San Francisco,884,222
1,MA,25025,Suffolk,797,214
2,NE,31007,Banner,0,1
3,NC,37181,Vance,44,26
4,TX,48421,Sherman,3,2


### Pull in and process the county and state-level shapefiles

In [4]:

last_year = datetime.datetime.now().year - 1

# State boundaries shape file
url_s = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_state_20m.zip'.format(last_year, last_year)

# Lower resolution county boundary shape file
url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_20m.zip'.format(last_year, last_year)

# Higher resolution coumnty shape files
#url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_5m.zip'.format(last_year, last_year)
#url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_500k.zip'.format(last_year, last_year)

response = requests.get(url)
response_s = requests.get(url_s)

# County shape file
with zipfile.ZipFile(StringIO(response.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
          
        elif ext == '.dbf':

            dbf = StringIO(z.read(fname))             
        else:
            pass
        
sf = shapefile.Reader(shp=shp, dbf=dbf)

# Drop county results before starting state processing
del shp, dbf

# State shape file
with zipfile.ZipFile(StringIO(response_s.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
    
        elif ext == '.dbf':
    
            dbf = StringIO(z.read(fname))             
        else:
            pass
        
sf_s = shapefile.Reader(shp=shp, dbf=dbf)

del shp, dbf

In [5]:
sf_s.fields

[('DeletionFlag', 'C', 1, 0),
 ['STATEFP', 'C', 2, 0],
 ['STATENS', 'C', 8, 0],
 ['AFFGEOID', 'C', 11, 0],
 ['GEOID', 'C', 2, 0],
 ['STUSPS', 'C', 2, 0],
 ['NAME', 'C', 100, 0],
 ['LSAD', 'C', 2, 0],
 ['ALAND', 'N', 14, 0],
 ['AWATER', 'N', 14, 0]]

In [6]:
sf_s.shapeRecords()[2].record

['08',
 '01779779',
 '0400000US08',
 '08',
 'CO',
 'Colorado',
 '00',
 268425964573,
 1178495763]

### Process the shape file data into a list of dictionaries
The dictionaries will include keys such as fips code and county / state names, as well as lists of lat/long coordinates. These can then be used to make any county-level map

In [7]:
final_county_data = []

for shprec in sf.shapeRecords():
    geo_dict = {}
    
    # Get fips, state abbr and name
    geo_dict['fips_state'] = shprec.record[0]
    geo_dict['fips_county'] = shprec.record[1]
    geo_dict['fips_combined'] = shprec.record[4]
    geo_dict['county_shortnm'] = shprec.record[5]
    
    lat, lon = map(list, zip(*shprec.shape.points))
    lat = [l if l < 0 else l-360 for l in lat]
    
    indices = shprec.shape.parts.tolist()
    
    lat = [lat[i:j] + ['null'] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + ['null'] for i, j in zip(indices, indices[1:]+[None])]
    
    geo_dict['lat'] = list(itertools.chain.from_iterable(lat))
    geo_dict['lon'] = list(itertools.chain.from_iterable(lon))

    final_county_data.append(geo_dict)
    
    
final_state_data = []

for shprec in sf_s.shapeRecords():
    geo_dict = {}
    
    # Get fips, state abbr and name
    geo_dict['fips_state'] = shprec.record[0]
    geo_dict['state_abbr'] = shprec.record[4]
    
    lat, lon = map(list, zip(*shprec.shape.points))
    lat = [l if l < 0 else l-360 for l in lat]
    
    indices = shprec.shape.parts.tolist()
    
    lat = [lat[i:j] + ['null'] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + ['null'] for i, j in zip(indices, indices[1:]+[None])]
    
    geo_dict['lat_state'] = list(itertools.chain.from_iterable(lat))
    geo_dict['lon_state'] = list(itertools.chain.from_iterable(lon))

    final_state_data.append(geo_dict)

### Convert to Pandas DataFrames of geography codes and lat / long coordinates

In [8]:
# To keep only continental U.S.
keep = ['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'ID',
'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 
'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
'WI', 'WY']

county_df = pd.DataFrame(final_county_data)

# Keep only continental US
state_df = pd.DataFrame(final_state_data)
state_df = state_df[state_df.state_abbr.isin(keep)]

county_df.head()

,county_shortnm,fips_combined,fips_county,fips_state,lat,lon
0,San Francisco,06075,075,06,"[-122.511983, -122.465396, -122.398139, -122.3...","[37.77113, 37.800879, 37.80563, 37.790724, 37...."
1,Suffolk,25025,025,25,"[-71.191155, -71.156887, -71.167625, -71.11719...","[42.283059, 42.330241, 42.360073, 42.364229, 4..."
2,Banner,31007,007,31,"[-104.052825235, -103.370391, -103.369024, -10...","[41.6979538531, 41.69921, 41.437655, 41.394633..."
3,Vance,37181,181,37,"[-78.497783, -78.4572778963, -78.3237185096, -...","[36.514477, 36.5414487084, 36.5424213916, 36.3..."
4,Sherman,48421,421,48,"[-102.162463, -102.032339019, -101.826565, -10...","[36.500326, 36.500065673, 36.499654, 36.499528..."


### Join in the population data

In [9]:
county_pop_df = pd.merge(county_df, pop_bins[['fips_combined', 'state_abbrev', 'population', 'bin']], 
                         on="fips_combined", how="inner")

# Retain only coninental US
county_pop_df = county_pop_df[county_pop_df.state_abbrev.isin(keep)]
county_pop_df.head()

,county_shortnm,fips_combined,fips_county,fips_state,lat,lon,state_abbrev,population,bin
0,San Francisco,06075,075,06,"[-122.511983, -122.465396, -122.398139, -122.3...","[37.77113, 37.800879, 37.80563, 37.790724, 37....",CA,884,222
1,Suffolk,25025,025,25,"[-71.191155, -71.156887, -71.167625, -71.11719...","[42.283059, 42.330241, 42.360073, 42.364229, 4...",MA,797,214
2,Banner,31007,007,31,"[-104.052825235, -103.370391, -103.369024, -10...","[41.6979538531, 41.69921, 41.437655, 41.394633...",NE,0,1
3,Vance,37181,181,37,"[-78.497783, -78.4572778963, -78.3237185096, -...","[36.514477, 36.5414487084, 36.5424213916, 36.3...",NC,44,26
4,Sherman,48421,421,48,"[-102.162463, -102.032339019, -101.826565, -10...","[36.500326, 36.500065673, 36.499654, 36.499528...",TX,3,2


## Generate the chloropleth map
Colorcet pallets available here: https://bokeh.github.io/colorcet/  
Color gradient generator: http://www.herethere.net/~samson/php/color_gradient/  
Color brewer: http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3

#### Colors and data setup

In [17]:
palette = my_colors.purple
#palette = cc.bgy (from colorcet)
#palette = viridis(9) (from Bokeh to generate up to 256 grades)

# Each time this is run the color order will reverse, chaning the look of the chart
palette.reverse()

color_mapper = LinearColorMapper(palette=palette, low=1, high=256) # Numbers correspond to min/max bins

# Create the column data sources
cds = ColumnDataSource(county_pop_df)
cds_state = ColumnDataSource(state_df)

#### Plot configuration

In [18]:
# Set the figure parameters and generate a figure
plot_size_and_tools = {'width': 900, 'height': 500, 'toolbar_location': None}

p = figure(title="Population in thousands by county", **plot_size_and_tools)

# Create a plot from the column data source
county_patches = p.patches('lat', 'lon', source=cds, line_color='grey', line_width=0.2, 
                           fill_color={'field': 'bin', 'transform': color_mapper})

# Add hover tool categories and values
hover = HoverTool(renderers=[county_patches], tooltips=[("County", "@county_shortnm"),
                                                        ("State", "@state_abbrev"),
                                                        ("FIPS", "@fips_combined"),
                                                        ("Population", "@population{0,0}")])

# Overlap state map boundaries
p.patches('lat_state', 'lon_state', source=cds_state, line_color="grey", fill_color=None)


# Add the color bar legend
ticker_bins = [1, 50, 100, 150, 200, 256]
ticker_overrides = {}
for bin in ticker_bins:
    ticker_overrides[bin] = "{0:,}".format(pop_bin_max[bin])

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=12, 
                     border_line_color=None, 
                     location=(40,50), 
                     height=150,
                     ticker=FixedTicker(ticks=ticker_bins),
                     formatter=NumeralTickFormatter(),
                     major_tick_line_color='white' ,
                     major_label_overrides=ticker_overrides)

# Add an optional tex label
text_to_show = "Ths. of people"
mytext = Label(x=30, y=80, x_units='screen', y_units='screen',
                 text=text_to_show, text_font_size="8pt", render_mode='css',
                 border_line_color=None, background_fill_color=None)

# Adjust plot settings
p.add_layout(mytext)
p.add_layout(color_bar, 'left')
p.add_tools(hover)
p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None
p.title.align = "center"

# Some other useful plot options...
"""
show(row(p1, p1)) - shows plots side by side
grid = gridplot([p1, p2], ncols=2, plot_width=800, plot_height=600) - accomplishes same as above

save(p, filename="/Users/mlc067/Documents/AdHoc/claims_legal_pop/plot_pop.html", 
         resources=None, title=None, state=None)
         
This will cause plots to be written to the file each time any plot is generated: 
output_file(filename="/Users/mlc067/Documents/AdHoc/claims_legal_pop/plot.html")
""";

In [19]:
show(p)

#### Log scale version
Makes the data appear more uniform by squashing larger values

In [20]:
# Note that you can't have negative numbers or zeros on a log scale
max_pop = county_pop_df.population.max()
color_mapper_log = LogColorMapper(palette=palette, low=1, high=max_pop)

plot_size_and_tools = {'width': 900, 'height': 500, 'toolbar_location': None}

p = figure(title="Population in thousands by county", **plot_size_and_tools)

# Create a plot from the column data source
county_patches = p.patches('lat', 'lon', source=cds, line_color='grey', line_width=0.2, 
                           fill_color={'field': 'population', 'transform': color_mapper_log})

# Add hover tool categories and values
hover = HoverTool(renderers=[county_patches], tooltips=[("County", "@county_shortnm"),
                                                        ("State", "@state_abbrev"),
                                                        ("FIPS", "@fips_combined"),
                                                        ("Population", "@population")])

# Overlap state map boundaries
p.patches('lat_state', 'lon_state', source=cds_state, line_color="grey", fill_color=None)


# Add the color bar legend
color_bar_log = ColorBar(color_mapper=color_mapper_log,
                     label_standoff=12, 
                     border_line_color=None, 
                     location=(40,50), 
                     height=150,
                     ticker=LogTicker(desired_num_ticks=6), # Wont show up if there are negatives or zeros in data
                     formatter=NumeralTickFormatter(),
                     major_tick_line_color='white')


# Adjust plot settings
p.add_layout(color_bar_log, 'left')
p.add_tools(hover)
p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None
p.title.align = "center"

In [21]:
show(p)